In [22]:
import pandas as pd 
import numpy as np
import os

os.chdir(r'c:/Users/Neel Patel/Documents/Github Repositories/PytorchAudio_Emotion_Classifier')
os.getcwd()

'c:\\Users\\Neel Patel\\Documents\\Github Repositories\\PytorchAudio_Emotion_Classifier'

In [24]:
data_path = r'Data/metadata-and-augmentations/'
SEED = 42
#Get the validation features (unaugmented training 20%)
val_training_df = pd.read_csv(data_path+'training_df.csv')

#GET RID OF THE SURPRISED CATEGORY!
val_training_df = val_training_df.loc[val_training_df['Emotion'] != 'surprised']

#Get random 20% of this data with equal balance of emotions!
#Function to sample 20% of data with equal balance of emotions
def sample_balanced(df, fraction=0.2):
    # Group by Emotion
    grouped = df.groupby('Emotion')

    # Sample fraction of each group and concatenate
    sampled_df = grouped.apply(lambda x: x.sample(frac=fraction, random_state=SEED)).reset_index(drop=True)
    return sampled_df

# Get the validation dataset
val_training_df = sample_balanced(val_training_df, fraction=0.2)

val_training_df['Emotion'].value_counts()


angry      254
disgust    254
fear       254
happy      254
sad        254
neutral    249
Name: Emotion, dtype: int64

In [ ]:
val_training_df.groupby('Emotion').apply(lambda x: x.sample)

In [28]:
#Load metadatas for train,val,test
metadata_train = np.load('Data/' + 'training_metadata_no_sur.npy',allow_pickle=True)
metadata_val = np.load('Data/' + 'val_metadata_no_sur.npy',allow_pickle=True)
#metadata_test = np.load('Data/' + 'test_metadata.npy', allow_pickle=True)

#Load HDF5 format files 
train_hdf5_file = 'Data/training_data.hdf5'
val_hdf5_file = 'Data/test_data.hdf5'
#test_hdf5_file = 'Data/test_data.hdf5'

train_metadata_df = pd.DataFrame(metadata_train)
val_metadata_df = pd.DataFrame(metadata_val)



In [29]:
train_metadata_df

,Filename,Filepath,Gender,Emotion
0,1007_IOM_NEU_XX_augmented_training_V1_1025.wav,augmented_training_samples/1007_IOM_NEU_XX_aug...,female,neutral
1,DC_n20_augmented_training_V1_243.wav,augmented_training_samples/DC_n20_augmented_tr...,male,neutral
2,1027_ITS_NEU_XX_augmented_training_V1_5727.wav,augmented_training_samples/1027_ITS_NEU_XX_aug...,male,neutral
3,03-01-02-01-02-01-05_augmented_training_V1_167...,augmented_training_samples/03-01-02-01-02-01-0...,male,neutral
4,1084_IEO_NEU_XX_augmented_training_V1_3757.wav,augmented_training_samples/1084_IEO_NEU_XX_aug...,female,neutral
...,...,...,...,...
7593,1042_DFA_ANG_XX_augmented_training_V1_5556.wav,augmented_training_samples/1042_DFA_ANG_XX_aug...,male,angry
7594,1080_IWL_ANG_XX_augmented_training_V1_6448.wav,augmented_training_samples/1080_IWL_ANG_XX_aug...,male,angry
7595,1071_IEO_ANG_MD_augmented_training_V1_3935.wav,augmented_training_samples/1071_IEO_ANG_MD_aug...,male,angry
7596,1008_IEO_ANG_MD_augmented_training_V1_7436.wav,augmented_training_samples/1008_IEO_ANG_MD_aug...,female,angry


In [30]:
val_metadata_df

,Filename,Filepath,Gender,Emotion
0,1022_IWL_ANG_XX.wav,Data\speech-emotion-recognition-en\Crema\1022_...,male,angry
1,1003_DFA_ANG_XX.wav,Data\speech-emotion-recognition-en\Crema\1003_...,female,angry
2,03-01-05-01-01-02-04.wav,Data\speech-emotion-recognition-en\Ravdess\aud...,female,angry
3,YAF_kite_angry.wav,Data\speech-emotion-recognition-en\Tess\YAF_an...,female,angry
4,OAF_page_angry.wav,Data\speech-emotion-recognition-en\Tess\OAF_an...,female,angry
...,...,...,...,...
1514,1011_IWL_SAD_XX.wav,Data\speech-emotion-recognition-en\Crema\1011_...,male,sad
1515,OAF_thumb_sad.wav,Data\speech-emotion-recognition-en\Tess\OAF_Sa...,female,sad
1516,1050_ITH_SAD_XX.wav,Data\speech-emotion-recognition-en\Crema\1050_...,male,sad
1517,1089_ITS_SAD_XX.wav,Data\speech-emotion-recognition-en\Crema\1089_...,female,sad


In [12]:
#Construct the model for audio classification

# Import packages!

import os
import shutil
from zipfile import ZipFile
import time
import random
from tqdm import tqdm

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import scipy
import librosa

from IPython.display import Audio

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torchaudio
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import torchaudio.transforms as T
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.model_selection import train_test_split

from utilities import Emotion_Classification_Waveforms
from modeling import MelSpec_CNN_Model,Feature_MLP_Model,CombinedModel
import torch.multiprocessing as mp

os.chdir(r'C:\Users\Neel Patel\Documents\Github Repositories\PytorchAudio_Emotion_Classifier')

#Load metadatas for train,val,test
metadata_train = np.load('Data/' + 'training_metadata_no_sur.npy',allow_pickle=True)
metadata_val = np.load('Data/' + 'val_metadata_no_sur.npy',allow_pickle=True)
#metadata_test = np.load('Data/' + 'test_metadata.npy', allow_pickle=True)

#Load HDF5 format files 
train_hdf5_file = 'Data/training_data_no_sur.hdf5'
val_hdf5_file = 'Data/validation_data_no_sur.hdf5'
#test_hdf5_file = 'Data/test_data.hdf5'

train_metadata_df = pd.DataFrame(metadata_train)
val_metadata_df = pd.DataFrame(metadata_val)
#test_metadata_df = pd.DataFrame(metadata_test)

# Create datasets
full_train_dataset = Emotion_Classification_Waveforms(
    hdf5_file_path=train_hdf5_file,
    metadata_df=train_metadata_df
)

val_dataset = Emotion_Classification_Waveforms(
    hdf5_file_path = val_hdf5_file,
    metadata_df=val_metadata_df
)

# Create DataLoaders for training and validation and testing
train_dataloader = DataLoader(full_train_dataset, batch_size=16, shuffle=True, num_workers=4,persistent_workers=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4,persistent_workers=True)


for i, batch in tqdm(enumerate(val_dataloader)):
    print(f"Batch {i}:")
    print(f"Mel Spectrogram shape: {batch['waveform_data']['Mel Spectrogram'].shape}")
    print(f"Features shape: {batch['waveform_data']['Features'].shape}")

    mel_spec = batch['waveform_data']['Mel Spectrogram']
    features = batch['waveform_data']['Features']

    labels = batch['emotion']

    print(batch['waveform_data']['Mel Spectrogram'].shape)
    break
    #if torch.isnan(labels).any() or torch.isinf(labels).any():
    #     print(f"NaNs/Infs found in labels at batch {i}")
    #     raise NotImplementedError(f"You have nans! at batch{i} for in these files {labels}")
    
    # if torch.isnan(mel_spec).any() or torch.isinf(mel_spec).any():
    #     raise NotImplementedError(f"You have nans! at batch{i} for in these files {batch['filename']}")
    #     #print("Found NaN or Inf in Mel Spectrogram!")
    # if torch.isnan(features).any() or torch.isinf(features).any():
    #     raise NotImplementedError(f"You have nans! at batch{i} for in these files {batch['filename']}")
    #     #print("Found NaN or Inf in Features!")

0it [00:01, ?it/s]

Batch 0:
Mel Spectrogram shape: torch.Size([16, 1, 128, 573])
Features shape: torch.Size([16, 302])
torch.Size([16, 1, 128, 573])


In [8]:
import h5py

with h5py.File('Data/training_data_no_sur.hdf5', 'r') as hdf:
    mel_spec = hdf['mel_spectrograms'][7306]
    features = hdf['features'][7306]

print(f"Mel Spectrogram: {mel_spec}")
print(f"Features: {features}")


Mel Spectrogram: [[[0.0328729  0.01947049 0.00231159 ... 0.         0.         0.        ]
  [0.03935537 0.02577184 0.0074062  ... 0.         0.         0.        ]
  [0.09875096 0.08533566 0.05752949 ... 0.         0.         0.        ]
  ...
  [0.00170687 0.00156399 0.00125927 ... 0.         0.         0.        ]
  [0.00110118 0.00104161 0.00093399 ... 0.         0.         0.        ]
  [0.00125739 0.00134854 0.00133952 ... 0.         0.         0.        ]]]
Features: [-2.20388718e+02  1.01122417e+01  7.25783539e+00  1.42866349e+00
 -1.56546221e+01 -1.22359447e+01 -1.16007626e+00 -3.86246037e+00
 -8.11057854e+00  1.48686266e+00 -5.73151731e+00  2.94954985e-01
 -2.33529472e+00  1.21093750e-01  2.41699219e-01  3.94531250e-01
  5.25878906e-01  6.45019531e-01  7.25097656e-01  7.70507812e-01
  7.83691406e-01  7.89550781e-01  7.82226562e-01  7.77832031e-01
  7.73925781e-01  6.19628906e-01  4.63378906e-01  3.17382812e-01
  1.76757812e-01  1.79199219e-01  1.83593750e-01  1.68457031e-01
 

In [9]:

os.getcwd()
file_path = train_metadata_df.iloc[7306]['Filepath']
print(file_path)
waveform, sample_rate = torchaudio.load(r'Data/metadata-and-augmentations/'+file_path)

from utilities import extract_hnr,extract_mel_spectrogram,extract_mfcc,extract_rms,extract_zero_crossing_rate
from utilities import load_dataset

# Process waveform to extract features
mfcc = extract_mfcc(waveform)
zcr = extract_zero_crossing_rate(waveform)
hnr = extract_hnr(waveform)
rms = extract_rms(waveform)

print(f"MFCC: {mfcc}")
print(f"ZCR: {zcr}")
print(f"HNR: {hnr}")
print(f"RMS: {rms}")


augmented_training_samples/YAF_hole_angry_augmented_training_V1_1081.wav
MFCC: tensor([[-122.9263,   15.1686,   10.6831,    1.9926,  -24.1162,  -18.7849,
           -1.8305,   -5.8688,  -12.3965,    2.3329,   -8.7366,    0.5298,
           -3.5394]])
ZCR: tensor([0.1211, 0.2417, 0.3945, 0.5259, 0.6450, 0.7251, 0.7705, 0.7837, 0.7896,
        0.7822, 0.7778, 0.7739, 0.6196, 0.4634, 0.3174, 0.1768, 0.1792, 0.1836,
        0.1685, 0.1470, 0.1279, 0.1099, 0.0972, 0.0942, 0.0903, 0.0986, 0.0923,
        0.0752, 0.0635, 0.0513, 0.0791, 0.0947, 0.1143, 0.1167, 0.0854, 0.0684,
        0.0459, 0.0303, 0.0229, 0.0215, 0.0205, 0.0239, 0.0269, 0.0303, 0.0356,
        0.0386, 0.0405, 0.0464, 0.0479, 0.0503, 0.0640, 0.0708, 0.0806, 0.0859,
        0.0840, 0.0786, 0.0625, 0.0503, 0.0605, 0.0942, 0.1196, 0.1362, 0.1421,
        0.1343, 0.1514, 0.1846, 0.2290, 0.2471, 0.2407, 0.2422, 0.2163, 0.1787,
        0.1523, 0.1079, 0.0659, 0.0566, 0.0518, 0.0508, 0.0469, 0.0459, 0.0518,
        0.0518, 0.0562, 

In [14]:

# Print the PATH environment variable
print("System PATH environment variable:")
print(list(os.environ['PATH'].split(';')))

System PATH environment variable:
['c:\\Users\\Neel Patel\\AppData\\Local\\Programs\\Python\\Python311', 'c:\\Users\\Neel Patel\\AppData\\Roaming\\Python\\Python311\\Scripts', 'C:\\Program Files\\Oculus\\Support\\oculus-runtime', 'C:\\Windows\\system32', 'C:\\Windows', 'C:\\Windows\\System32\\Wbem', 'C:\\Windows\\System32\\WindowsPowerShell\\v1.0\\', 'C:\\Windows\\System32\\OpenSSH\\', 'C:\\Program Files\\NVIDIA Corporation\\NVIDIA NvDLISR', 'C:\\Program Files (x86)\\NVIDIA Corporation\\PhysX\\Common', 'C:\\Program Files\\Microsoft SQL Server\\Client SDK\\ODBC\\170\\Tools\\Binn\\', 'C:\\Program Files (x86)\\Microsoft SQL Server\\150\\Tools\\Binn\\', 'C:\\Program Files\\Microsoft SQL Server\\150\\Tools\\Binn\\', 'C:\\Program Files\\Microsoft SQL Server\\150\\DTS\\Binn\\', 'C:\\Program Files (x86)\\Windows Kits\\8.1\\Windows Performance Toolkit\\', 'C:\\Program Files\\MATLAB\\R2022b\\bin', 'C:\\WINDOWS\\system32', 'C:\\WINDOWS', 'C:\\WINDOWS\\System32\\Wbem', 'C:\\WINDOWS\\System32\\Wind